In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

data_path = "/content/drive/My Drive/Colab Notebooks/"


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime
from yellowbrick.regressor import ResidualsPlot
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


def plot_actuals(x, y, ticker):
    figure = plt.figure()
    figure.title = "Start prices of " + ticker
    plt.plot(x, y)
    plt.xlabel("Time")
    plt.ylabel("Starting Price")
    plt.show()
    # plt.savefig(ticker + ".png")


def run_linear_regression(X_train, X_test, y_train, y_test):
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    predictions = regressor.predict(X_test)

    r2 = r2_score(y_test, predictions)
    print("R2 for " + ticker + ": " + str(r2))

    mse = mean_squared_error(y_test, predictions)
    print("MSE for " + ticker + ": " + str(mse))

    return mse


df = pd.read_csv(data_path + "cooked_v3.csv")
print(df.head())
print(df.CalcDateTime.min())
print(df.CalcDateTime.max())

# tickers and their traded volumes
total_volume = df['TradedVolume'].sum()
volume_series = df.groupby('Mnemonic')['TradedVolume'].sum()
volume_series.sort_values(ascending=False, inplace=True)
print(volume_series)

df.head()
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)

start_time = datetime.now()
test_size = 0.2
n_splits = int((1//test_size)-1)

total_weighted_mse = 0
unweighted_mse = 0
for ticker in volume_series.index:
    ticker_data = df[df['Mnemonic'] == ticker]
    vol = ticker_data['TradedVolume'].sum()
    ticker_data.reset_index(inplace=True)

    X = pd.to_numeric(pd.to_datetime(ticker_data['CalcDateTime']))
    X = X.values.reshape(-1, 1)

    Y = ticker_data['StartPrice']
    print("Ticker " + ticker + " has " + str(len(X)) + " records.")

    # plot hourly since min-by-min is really slow
    ticker_data['CalcDateTime'] = pd.to_datetime(ticker_data['CalcDateTime'])


    # predict last 15 values since arima on 25% of the data is taking too long
    train_index = np.arange(0,len(X) - 15)
    test_index = np.arange(len(X) - 15, len(X))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]


    # ############## LINEAR REGRESSION ###################
    mse = run_linear_regression(X_train, X_test, y_train, y_test)
    weighted_mse = mse*(vol/total_volume)
    print(weighted_mse)
    total_weighted_mse = total_weighted_mse + weighted_mse

    unweighted_mse = unweighted_mse + mse
    print(unweighted_mse)


end_time = datetime.now()
c = end_time - start_time
print("Ran LR in:" + str(c.total_seconds()))

print("Weighted MSE: " + str(total_weighted_mse))
print("Average MSE: " + str(unweighted_mse/100)) # sum/# of tickers

          CalcDateTime Mnemonic  MinPrice  MaxPrice  StartPrice  EndPrice  \
0  2017-07-03 08:00:00      SNH     4.450     4.456       4.456     4.450   
1  2017-07-03 08:01:00      SNH     4.450     4.456       4.456     4.450   
2  2017-07-03 08:02:00      SNH     4.450     4.456       4.456     4.450   
3  2017-07-03 08:03:00      SNH     4.457     4.457       4.457     4.457   
4  2017-07-03 08:04:00      SNH     4.457     4.457       4.457     4.457   

   HasTrade  TradedVolume  NumberOfTrades  
0       1.0        5505.0             9.0  
1       0.0           0.0             0.0  
2       0.0           0.0             0.0  
3       1.0         899.0             2.0  
4       0.0           0.0             0.0  
2017-07-03 08:00:00
2018-05-15 20:00:00
Mnemonic
SNH     4.182139e+09
DBK     2.346360e+09
EOAN    1.883320e+09
DTE     1.695683e+09
CBK     1.575910e+09
RWE     8.374443e+08
IFX     7.630757e+08
SVAB    6.026515e+08
LHA     5.934044e+08
DAI     4.894308e+08
O2D     4.2069

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ticker DBK has 156457 records.
R2 for DBK: 0.0
MSE for DBK: 1.1655600979710683
0.10735000977475653
1.9919520596133369


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ticker EOAN has 156457 records.
R2 for EOAN: -3.3865516915701953e+28
MSE for EOAN: 0.10686072931959252
0.00789978184205284
2.0988127889329293


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ticker CAP has 156447 records.
R2 for CAP: 0.0
MSE for CAP: 0.033865541632411095
4.6182685965833756e-05
1128.3173632255753


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ticker 3W9K has 156034 records.
R2 for 3W9K: -1.803305660837438e+32
MSE for 3W9K: 0.0021706502319311467
2.827575029761204e-06
1128.319533875807
Ran LR in:85.716668
Weighted MSE: 6.474657841210102
Average MSE: 11.283195338758071


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
